# BAIT 509 Assignment 3 

__Name__: Liqiong Sun

__Student_number__: 75956961

# Exercise 4 (30%): Fitting SVM

For this exercise, you're expected to use the auto_data.csv dataset, not the mtcars data set. The name and mpg columns should not be used as predictors. Your task is to use SVM to predict whether a car has high or low mileage (column mileage) using two methods:

1. An SVM with a linear kernel (so, a basic support vector classifier)
2. An SVM with a radial basis kernel

Answer the following questions:

1. Split the data into random training and validation sets. Set aside 40% of the data for the validation set.
2. Fit the two models over a grid of hyperparameters, and report the validation error in all cases. You can report them in a table, or a plot. To make things easy, you don't have to choose a fine grid, as long as the optimal hyperparameters are located somewhere in the range of the grid.
3. For both methods, what hyperparameter(s) on your grid has the best validation error? Of these two models, which one is better, the linear or radial?

__Answer to 3__ : For linear model, when c = 10, it has lowest validation error of 0.102. For radial basis model, when gamma=1e-05 and c =100, it has lowest validation error of 0.096. In conclusion, linear model is better as it is more stable than radial basis model and it have overall smaller validation error.

In [32]:
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import scale

df1 = pd.read_csv('./auto_data.csv')
print(df1)

      mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0    18.0          8         307.0         130  3504.0          12.0    70   
1    15.0          8         350.0         165  3693.0          11.5    70   
2    18.0          8         318.0         150  3436.0          11.0    70   
3    16.0          8         304.0         150  3433.0          12.0    70   
4    17.0          8         302.0         140  3449.0          10.5    70   
5    15.0          8         429.0         198  4341.0          10.0    70   
6    14.0          8         454.0         220  4354.0           9.0    70   
7    14.0          8         440.0         215  4312.0           8.5    70   
8    14.0          8         455.0         225  4425.0          10.0    70   
9    15.0          8         390.0         190  3850.0           8.5    70   
10   15.0          8         383.0         170  3563.0          10.0    70   
11   14.0          8         340.0         160  3609.0          

In [33]:
X = df1.drop(["mileage","mpg","name"], axis=1)
Y = df1["mileage"]

In [34]:
X_scaled = scale(X)
training_X, testing_X, training_Y, testing_Y = train_test_split(X_scaled, Y, test_size=0.4, random_state=1, shuffle=False)

In [35]:
tuned_parameters = {'kernel': ['rbf', 'linear'], 'gamma': [1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8],
                     'C': [1, 10, 100, 1000]}

In [39]:
res = {}
for k in tuned_parameters['kernel']:
    res[k] = {}
    for c in tuned_parameters['C']:
        if k == 'linear':
            # gamma is not for linear kernel
            clf = svm.SVC(kernel=k, C=c)
            clf.fit(training_X, training_Y)
            test_error = clf.score(testing_X, testing_Y)
#             pred = clf.predict(testing_X)
            #test_error = np.sum(np.abs(pred-testing_Y)) / len(testing_Y)
            res[k][c] = 1-test_error
            print('{:.3f} for {{kernel: {}, C: {}}}'.format(res[k][c], k, c))
            continue
        res[k][c] = {}
        for g in tuned_parameters['gamma']:
            clf = svm.SVC(kernel=k, C=c, gamma=g)
            clf.fit(training_X, training_Y)
            test_error = clf.score(testing_X, testing_Y)
#             pred = clf.predict(testing_X)
#             test_error = np.sum(np.abs(pred-testing_Y)) / len(testing_Y)
            res[k][c][g] = 1-test_error
            print('{:.3f} for {{kernel: {}, C: {}, gamma: {}}}'.format(res[k][c][g], k, c, g))

0.102 for {kernel: rbf, C: 1, gamma: 0.001}
0.732 for {kernel: rbf, C: 1, gamma: 0.0001}
0.732 for {kernel: rbf, C: 1, gamma: 1e-05}
0.732 for {kernel: rbf, C: 1, gamma: 1e-06}
0.732 for {kernel: rbf, C: 1, gamma: 1e-07}
0.732 for {kernel: rbf, C: 1, gamma: 1e-08}
0.115 for {kernel: rbf, C: 10, gamma: 0.001}
0.102 for {kernel: rbf, C: 10, gamma: 0.0001}
0.732 for {kernel: rbf, C: 10, gamma: 1e-05}
0.732 for {kernel: rbf, C: 10, gamma: 1e-06}
0.732 for {kernel: rbf, C: 10, gamma: 1e-07}
0.732 for {kernel: rbf, C: 10, gamma: 1e-08}
0.115 for {kernel: rbf, C: 100, gamma: 0.001}
0.115 for {kernel: rbf, C: 100, gamma: 0.0001}
0.096 for {kernel: rbf, C: 100, gamma: 1e-05}
0.732 for {kernel: rbf, C: 100, gamma: 1e-06}
0.732 for {kernel: rbf, C: 100, gamma: 1e-07}
0.732 for {kernel: rbf, C: 100, gamma: 1e-08}
0.115 for {kernel: rbf, C: 1000, gamma: 0.001}
0.115 for {kernel: rbf, C: 1000, gamma: 0.0001}
0.115 for {kernel: rbf, C: 1000, gamma: 1e-05}
0.096 for {kernel: rbf, C: 1000, gamma: 1e-06

In [40]:
# formating validation error for linear kernel
print('{:_<18}'.format(''))
print('| {:<4} | {:<5} |'.format('linear', ''))
print('{:=<18}'.format(''))
for c in res['linear']:
    print('| C={:<4} | {:.3f} |'.format(c, res['linear'][c]))

print('{:_<100}'.format(''))
print('| {:<6} |'.format('rbf'), end='')
for g in tuned_parameters['gamma']:
    print(' gamma={:<6} |'.format(g), end='')
print()
print('{:=<100}'.format(''))
for c in res['rbf']:
    print('| C={:<4} |'.format(c), end='')
    for g in res['rbf'][c]:
         print(' {:<12.3f} |'.format(res['rbf'][c][g]), end='')
    print()

__________________
| linear |       |
| C=1    | 0.108 |
| C=10   | 0.102 |
| C=100  | 0.108 |
| C=1000 | 0.108 |
____________________________________________________________________________________________________
| rbf    | gamma=0.001  | gamma=0.0001 | gamma=1e-05  | gamma=1e-06  | gamma=1e-07  | gamma=1e-08  |
| C=1    | 0.102        | 0.732        | 0.732        | 0.732        | 0.732        | 0.732        |
| C=10   | 0.115        | 0.102        | 0.732        | 0.732        | 0.732        | 0.732        |
| C=100  | 0.115        | 0.115        | 0.096        | 0.732        | 0.732        | 0.732        |
| C=1000 | 0.115        | 0.115        | 0.115        | 0.096        | 0.732        | 0.732        |


In [42]:
# from sklearn.model_selection import PredefinedSplit
# test_fold_idx = np.zeros(len(X))
# for i in range(int(len(X) * 0.6)):
#     test_fold_idx[i] = -1

# cv = PredefinedSplit(test_fold_idx)
# scores = ['precision']

# for score in scores:
#     print("# Tuning hyper-parameters for %s" % score)
#     print()

#     clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=cv,
#                        scoring='%s_macro' % score)
#     clf.fit(X, Y)
  
#     print("Best parameters set found on validation set:")
#     print()
#     print(clf.best_params_)
#     print()
#     print("Grid scores on validation set:")
#     print()
#     means = clf.cv_results_['mean_test_score']
#     stds = clf.cv_results_['std_test_score']
#     for mean, std, params in zip(means, stds, clf.cv_results_['params']):
#         print("%0.3f (+/-%0.03f) for %r"
#               % (mean, std * 2, params))
#     print()

#     print("Detailed classification report:")
#     print()
#     print("The model is trained on the full development set.")
#     print("The scores are computed on the full evaluation set.")
#     print()
#     testing_Y, predicting_Y = testing_Y, clf.predict(testing_X)
#     print(classification_report(testing_Y, predicting_Y))
#     print()

# Tuning hyper-parameters for precision

Best parameters set found on validation set:

{'C': 1, 'gamma': 0.001, 'kernel': 'linear'}

Grid scores on validation set:

0.764 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.847 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
0.779 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.847 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
0.807 (+/-0.000) for {'C': 1, 'gamma': 1e-05, 'kernel': 'rbf'}
0.847 (+/-0.000) for {'C': 1, 'gamma': 1e-05, 'kernel': 'linear'}
0.774 (+/-0.000) for {'C': 1, 'gamma': 1e-06, 'kernel': 'rbf'}
0.847 (+/-0.000) for {'C': 1, 'gamma': 1e-06, 'kernel': 'linear'}
0.783 (+/-0.000) for {'C': 1, 'gamma': 1e-07, 'kernel': 'rbf'}
0.847 (+/-0.000) for {'C': 1, 'gamma': 1e-07, 'kernel': 'linear'}
0.804 (+/-0.000) for {'C': 1, 'gamma': 1e-08, 'kernel': 'rbf'}
0.847 (+/-0.000) for {'C': 1, 'gamma': 1e-08, 'kernel': 'linear'}
0.766 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'r

In [25]:
# kernels = ['rbf', 'linear']
# validation_error_kernel = {'rbf': [], 'linear': []}
# means = clf.cv_results_['mean_test_score']
# for mean, params in zip(means, clf.cv_results_['params']):
#     validation_error_kernel[params['kernel']].append((1-mean, params['C'], params['gamma']))
#     #print("%0.3f for %r" % (1-mean, params))
# #print()